In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# import holt winters
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES

# import sarimax
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [17]:
df = pd.read_csv('electrecity.csv', index_col='Month', parse_dates=True)
# rename Month column to Date
df.index.names = ['Date']
df.dropna(inplace=True)
df = df.iloc[::-1]
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15996\4100388942.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('electrecity.csv', index_col='Month', parse_dates=True)


,Sales
Date,
2000-12-01,310816.06880
2001-01-01,272334.73804
2001-02-01,269298.40142
2001-03-01,254390.93545
2001-04-01,264490.49166


In [18]:
train_size = len(df) - 72

In [19]:
mse_hwes = []
for i in range (5):
    start_index = 12 * i
    end_index = start_index + train_size + 12
    train = df.iloc[start_index:train_size + start_index]
    test = df.iloc[train_size + start_index:end_index]

    hwes_model = HWES(
    train,
    trend='mul',
    damped_trend=True,
    seasonal='mul',
    seasonal_periods=12
    ).fit()

    # calculate mean squared error
    mse_hwes.append(mean_squared_error(test, hwes_model.forecast(12)))

c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: O

In [20]:
sarimax_mse = []
for i in range (5):
    start_index = 12 * i
    end_index = start_index + train_size + 12
    train = df.iloc[start_index:train_size + start_index]
    test = df.iloc[train_size + start_index:end_index]

    model_sarimax = SARIMAX(train, order=(0,0,1), seasonal_order=(0,1,0,12))
    fit_SARIMAX = model_sarimax.fit()
    start = len(train)
    end = len(train) + len(test) -1
    prediction_SARIMA = fit_SARIMAX.predict(start, end).rename('SARIMA')

    sarimax_mse.append(mean_squared_error(test, prediction_SARIMA))

c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Python312\Lib\site-packages

In [21]:
# make table for mse results for each window and model
results = pd.DataFrame({'HWES': mse_hwes, 'SARIMAX': sarimax_mse})

In [22]:
results

,HWES,SARIMAX
0,6.594173e+07,1.776226e+08
1,3.248256e+07,6.101795e+07
2,8.854366e+09,1.254622e+08
3,3.484484e+07,1.100038e+08
4,1.672371e+08,1.538786e+08
